In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Tuple
from termcolor import colored

<div class="alert alert-warning" align="center"><b>DIMENSIONNEMENT DE L'EXOCET+</b></div>

In [54]:
## DONNÉES

P = 101325.0
T = 288.0
r_air = 8.314/(28.966*1e-3)
gamma = 1.4
Cd_c = 0.4
Mach_cruise = 2.0
Mach_acc = 0.6
g0 = 9.80665
v_son = np.sqrt(gamma * r_air * T)
distance_max = 100 * 1e3

<div class="alert alert-info" align="center">TEMPS DE MISSION</div>

In [55]:
Gamma = 10 * g0
t_acc = np.divide((Mach_cruise - Mach_acc) * v_son, Gamma)
print(f"Temps d'accélération du missile = {colored(t_acc, 'yellow')} s\n")

x_acc = 0.5 * Gamma * t_acc **2 + Mach_acc * v_son * t_acc
print(f"Distance parcourue pendant la phase d'accélération = {colored(x_acc, 'yellow')} m\n")

t_cruise = (distance_max - x_acc) / (Mach_cruise * v_son)
print(f"Temps durant la phase de croisière = {colored(t_cruise, 'yellow')} s\n")

t_mission = t_acc + t_cruise
print(f"Temps de mission = {colored(t_mission, 'blue')} s\n")

if t_mission <= 200:
    print(f"{colored('La mission est réalisble', 'green')}")
else:
    print(f"{colored('La mission n est pas réalisable', 'red')}")

Temps d'accélération du missile = 4.85655564067031 s

Distance parcourue pendant la phase d'accélération = 2147.7945185679855 m

Temps durant la phase de croisière = 143.82010925319975 s

Temps de mission = 148.67666489387005 s

La mission est réalisble


<div class="alert alert-info" align="center">PHASE DE VOL</div>

In [56]:
## DONNÉES

Isp_c = 225.0
s = 1.1
i_c = 0.15
C_c = 0.75
rho_c = 1800.0

## FUNCTION

mass_cr = lambda m_cp: (i_c * m_cp) / (1 - i_c)

length_cp = lambda d_m: 8 * d_m

volume_cr = lambda m_cp, C_c, rho_c: np.divide(m_cp, C_c * rho_c)

length_cr = lambda d_m, V_cr: np.divide(4 * V_cr, np.pi * d_m**2)

def missile_diam(gamma:float, P_amb:float, Cd:float, M_cruise:float, t_cruise:float, rho:float, g:float, Isp:float, s:float) -> float:
    return s * np.divide(gamma * P_amb * Cd * M_cruise**2 * t_cruise, 16 * rho * g * Isp)

def mass_cruise_propergol(gamma:float, P_amb:float, Cd:float, M_cruise:float, s:float, g:float, Isp:float, t_cruise:float, d_mis:float) -> float:
    return s * np.divide(np.pi * gamma * P_amb * M_cruise**2 * Cd * d_mis**2, 8 * g * Isp) * t_cruise

## CODE
d_m = missile_diam(gamma=gamma, P_amb=P, Cd=Cd_c, M_cruise=Mach_cruise,
                   t_cruise=t_cruise, rho=rho_c, g=g0, Isp=Isp_c, s=s)
print(f"Diamètre du missile = {colored(d_m, 'yellow')} m\n")

m_cp = mass_cruise_propergol(gamma=gamma, P_amb=P, Cd=Cd_c, M_cruise=Mach_cruise, 
                             s=s, g=g0, Isp=Isp_c, t_cruise=t_cruise, d_mis=d_m)
print(f"Masse de propergol durant la phase de croisière = {colored(m_cp, 'yellow')} kg\n")

m_cr = mass_cr(m_cp=m_cp)
print(f"Masse du résevoir de propergol pendant la phase de croisière = {colored(m_cr, 'yellow')} kg\n")

L_cp = length_cp(d_m=d_m)
print(f"Longueur du propergol = {colored(L_cp, 'yellow')} m\n")

V_cr = volume_cr(m_cp=m_cp, C_c=C_c, rho_c=rho_c)
print(f"Volume du réservoir de propergol pendant la phase de croisère = {colored(V_cr, 'yellow')} m^3\n")

L_cr = length_cr(d_m=d_m, V_cr=V_cr)
print(f"Longueur du réservoir de propergol pendant la phase de vol = {colored(L_cr, 'yellow')} m^3\n")

Diamètre du missile = 0.5650426743788414 m

Masse de propergol durant la phase de croisière = 2040.3098199921535 kg

Masse du résevoir de propergol pendant la phase de croisière = 360.05467411626233 kg

Longueur du propergol = 4.520341395030731 m

Volume du réservoir de propergol pendant la phase de croisère = 1.5113406074015951 m^3

Longueur du réservoir de propergol pendant la phase de vol = 6.027121860040975 m^3



<div class="alert alert-info" align="center">PHASE D'ACCÉLÉRATION</div>

In [57]:
## DONNÉES

Isp_acc = 240.0
C_acc = 0.7
i_acc = 0.16
rho_acc = 1800.0

Delta_V = (Mach_cruise - Mach_acc) * v_son
m_ogive, m_equipement, m_payload = 25.0, 55.0, 200.0

## FUNCTION

mass_wings = lambda d_m: 20 + 200 * d_m**2

length_ap = lambda d_m, V_ap: np.divide(4 * V_ap, np.pi * d_m**2)

mass_ar = lambda m_ap, i_a: i_a * m_ap / (1 - i_a)

def mass_acc_prop(i_a:float, Isp:float, g:float, mass_empty:float, DeltaV:float) -> float:
    return (1 - i_a) * mass_empty * (np.exp(DeltaV / (Isp * g)) - 1)

## CODE
m_wings = mass_wings(d_m=d_m)
print(f"Masse des ailes = {colored(m_wings, 'yellow')} kg\n")

m_vide_arr = np.array([m_ogive, m_equipement, m_payload, m_wings, m_cp, m_cr])
m_vide = np.sum(m_vide_arr)
print(f"Masse à vide pour pendant la phase d'accélération = {colored(m_vide, 'red')} kg\n")

m_ap = mass_acc_prop(i_a=i_acc, Isp=Isp_acc, g=g0, mass_empty=m_vide, DeltaV=Delta_V)
print(f"Masse de propergol pour la phase d'accélération = {colored(m_ap, 'yellow')} kg\n")

V_ap = m_ap / rho_acc
print(f"Volume de propergol pour la phase d'accélération = {colored(V_ap, 'yellow')} m^3\n")

V_ar = V_ap / C_acc
print(f"Volume du réservoir de propergol pour la phase d'accélération = {colored(V_ar, 'yellow')} m^3\n")

L_ap = length_ap(d_m=d_m, V_ap=V_ap)
print(f"Longueur du propergol pour la phase d'accélération = {colored(L_ap, 'yellow')} m\n")

L_ar = L_ap / C_acc
print(f"Longueur du réservoir d'ergol pour la phase d'accélération = {colored(L_ar, 'yellow')} m\n")

m_ar = mass_ar(m_ap=m_ap, i_a=i_acc)
print(f"Masse du réservoir de propergol lors de la phase d'accélération = {colored(m_ar, 'yellow')} kg\n")

Masse des ailes = 83.85464477383869 kg

Masse à vide pour pendant la phase d'accélération = 2764.219138882255 kg

Masse de propergol pour la phase d'accélération = 520.7757629607802 kg

Volume de propergol pour la phase d'accélération = 0.28931986831154455 m^3

Volume du réservoir de propergol pour la phase d'accélération = 0.4133140975879208 m^3

Longueur du propergol pour la phase d'accélération = 1.1537876335121398 m

Longueur du réservoir d'ergol pour la phase d'accélération = 1.6482680478744856 m

Masse du réservoir de propergol lors de la phase d'accélération = 99.195383421101 kg



<div class="alert alert-info" align="center">EXPRESSION DES CARACTÉRISTIQUES GLOBALES DU MISSILE</div>

In [58]:
## FUNCTION

volume = lambda mass, rho: np.divide(mass, rho)
length = lambda V, d_m: np.divide(4 * V, np.pi * d_m**2)

## DONNÉES
rho_eq = 1500
rho_payload = 3000

## CODE

L_ogive = 3 * d_m

m_tot_missile_arr = np.concatenate([m_vide_arr, [m_ap, m_ar]])
m_tot = np.sum(m_tot_missile_arr)

V_eq = volume(mass=m_equipement, rho=rho_eq)
V_payload = volume(mass=m_payload, rho=rho_payload)

L_eq = length(V=V_eq, d_m=d_m)
L_payload = length(V=V_payload, d_m=d_m)
print(f"Longueur :\n    - equipement : {colored(L_eq, 'blue')} m\n    - payload : {colored(L_payload, 'blue')} m\n    - ogive : {colored(L_ogive, 'blue')} m\n")

L_tot_arr = np.array([L_ogive, L_eq, L_payload, L_ar, L_cr])
L_tot = np.sum(L_tot_arr)
print(f"Longueur totale du missile EXOCET+ = {colored(L_tot, 'blue')} m\n")

print(f"Masse totale du missile EXOCET+ = {colored(m_tot, 'blue')} kg\n")

print(f"Diamètre du missile EXOCET+ = {colored(d_m, 'blue')} m\n")

Longueur :
    - equipement : 0.146224131819929 m
    - payload : 0.26586205785441636 m
    - ogive : 1.6951280231365242 m

Longueur totale du missile EXOCET+ = 9.78260412072633 m

Masse totale du missile EXOCET+ = 3384.1902852641356 kg

Diamètre du missile EXOCET+ = 0.5650426743788414 m

